In [ ]:
!pip install nltk

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
twit = pd.read_csv(r"C:\Users\sujee\OneDrive\Desktop\Dataset\Twitter_Data.csv")
twit.head()

In [ ]:
# Change dependent variable to categorical
twit['category'] = twit['category'].map({0.0: 'Neutral', -1.0: 'Negative', 1.0: 'Positive'})

In [ ]:
twit.head()

In [ ]:
# Drop null/missing values
twit.dropna(inplace=True)

In [ ]:
twit.isnull().sum()

In [ ]:
# Text cleaning function
def clean_text(text):
    # Remove symbols except alphanumeric
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Transform to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return ' '.join(filtered_text)

In [ ]:
nltk.download('stopwords')

In [ ]:
nltk.download('punkt')

In [ ]:
# Apply text cleaning
twit['cleaned_text'] = twit['clean_text'].apply(clean_text)

In [ ]:
twit.drop('clean_text', axis=1, inplace=True)

In [ ]:
# Create a new column for the length of each sentence
twit['sentence_length'] = twit['cleaned_text'].apply(lambda x: len(x.split()))

In [ ]:
twit.head()

In [ ]:
twit = twit[['cleaned_text','sentence_length','category']]
twit.head()

In [ ]:
# Split data into dependent (X) and independent (y) dataframes
X = twit['cleaned_text']
y = twit['category']

In [ ]:
# Do one-hot encoding for each sentence
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_encoded = tokenizer.texts_to_sequences(X)

In [ ]:
# Add padding from the front side
max_sequence_length = max(twit['sentence_length'])
X_padded = pad_sequences(X_encoded, maxlen=max_sequence_length, padding='pre')

In [ ]:
# Build LSTM model
vocabulary_size = len(tokenizer.word_index) + 1
embedding_dim = 128
lstm_units = 64

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabulary_size, embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.LSTM(lstm_units),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Do dummy variable creation for the dependent variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = tf.keras.utils.to_categorical(y_encoded)

In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_categorical, test_size=0.2, random_state=42)

In [ ]:
# Train the model
training = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

In [ ]:
# Make predictions on the test set
y_pred_prob = model.predict(X_test)

In [ ]:
# Normalize predictions
y_pred = np.round(y_pred_prob)

In [ ]:
# Convert predictions back to original categories
y_pred_category = label_encoder.inverse_transform(np.argmax(y_pred, axis=1))
y_true_category = label_encoder.inverse_transform(np.argmax(y_test, axis=1))

In [ ]:
# Print classification report
print(classification_report(y_true_category, y_pred_category))